In [2]:
from huggingface_hub import login
login()

In [3]:
from datasets import load_dataset
import json
from transformers import TextStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os


In [4]:
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
ds = load_dataset("AI-MO/minif2f_test", split="train")
save_path = os.path.expanduser("~/assip2025/data/minif2f_saved")
ds.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/244 [00:00<?, ? examples/s]

In [5]:
save_path = os.path.expanduser("~/assip2025/data/minif2f_saved")
from datasets import load_from_disk
ds = load_from_disk(save_path)

In [6]:
MODEL_ID = "RickyDeSkywalker/TheoremLlama"
tok  = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",         
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
model_save_path = os.path.expanduser("~/assip2025/saved_models/saved_theorem_llama")
tok.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

In [9]:
model_save_path = os.path.expanduser("~/assip2025/saved_models/saved_theorem_llama")
tok = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModelForCausalLM.from_pretrained(model_save_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
streamer = TextStreamer(tok)

In [11]:
SAVE_DIR = os.path.expanduser("~/assip2025/llama_proofs/LlamaProofs")

In [12]:
tok.pad_token = tok.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
if __name__ == "__main__":
    total_problems = len(ds)
    print(f"\n Starting ZERO-SHOT proof generation for {total_problems} problems...")

    for i, problem in enumerate(ds):
        # Extract data using the feature names from your dataset_info.json
        problem_name = problem.get("name")
        formal_statement = problem.get("formal_statement")
        informal_statement = problem.get("informal_prefix")

        if not problem_name or not formal_statement:
            print(f"Warning: Skipping problem at index {i} due to missing 'name' or 'formal_statement'.")
            continue

        print(f"\n[{i+1}/{total_problems}] Processing Problem: {problem_name}")

        # The zero-shot prompt is simply the formal statement itself.
        # It should ideally end with `:= by` to prompt the model for completion.
        prompt = (
        f"-- Formal statement \n{formal_statement}\n\n"
        )

        # --- Generate Proof ---
        # 1. Tokenize the prompt and move the resulting tensors to the same device as the model
        inputs = tok(prompt, return_tensors="pt").to(model.device)


        # 2. Generate text using the model
        print("  Generating proof...")
        with torch.no_grad(): # Disable gradient calculations for faster inference
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,      # Max length of the generated proof
                #temperature=0,         # Low temperature for more deterministic, less random output
                do_sample=False,
                eos_token_id=tok.eos_token_id,
                pad_token_id=tok.eos_token_id,
                streamer=streamer,
            )

        # 3. Decode only the newly generated tokens back into text
        completion = tok.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

        # 4. Combine the original statement with the model's completion for a full file
        full_proof_attempt = prompt + completion

        # --- Save the proof to a .lean file ---
        output_filename = os.path.join(SAVE_DIR, f"{problem_name}.lean")
        try:
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(full_proof_attempt)
            print(f"  ✅ Proof attempt saved to: {output_filename}")
        except IOError as e:
            print(f"  ❌ Error saving file {output_filename}: {e}")

    print(f"\n--- Generation Complete ---")
    print(f"✅ All {total_problems} proof attempts have been generated and saved in '{SAVE_DIR}'.")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



 Starting ZERO-SHOT proof generation for 244 problems...

[1/244] Processing Problem: mathd_algebra_478
  Generating proof...
<|begin_of_text|>-- Formal statement 
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The volume of a cone is given by the formula $V = \frac{1}{3}Bh$, where $B$ is the area of the base and $h$ is the height. The area of the base of a cone is 30 square units, and its height is 6.5 units. What is the number of cubic units in its volume? Show that it is 65.-/
theorem mathd_algebra_478 (b h v : ℝ) (h₀ : 0 < b ∧ 0 < h ∧ 0 < v) (h₁ : v = 1 / 3 * (b * h))
    (h₂ : b = 30) (h₃ : h = 13 / 2) : v = 65 := by


  have h₄ : v = 1 / 3 * (b * h) := h₁;
  have h₅ : b = 30 := h₂;
  have h₆ : h = 13 / 2 := h₃;
  have h₇ : v = 1 / 3 * (30 * (13 / 2)) := by
    rw [h₅, h₆];
  have h₈ : v = 1 / 3 * (30 * 13 / 2) := by
    rw [h₇];
  have h₉ : v = 1 / 3 * (15 * 13) := by
    rw [h₈];
  have h₁₀ : v = 1 / 3 * 195 := by
    rw [h

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  ✅ Proof attempt saved to: /root/assip2025/llama_proofs/LlamaProofs/mathd_algebra_478.lean

[2/244] Processing Problem: numbertheory_4x3m7y3neq2003
  Generating proof...
<|begin_of_text|>-- Formal statement 
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Show that there are no integers $x$ and $y$ such that $4x^3 - 7y^3 = 2003$.-/
theorem numbertheory_4x3m7y3neq2003 (x y : ℤ) : 4 * x ^ 3 - 7 * y ^ 3 ≠ 2003 := by


# Formal statement 
# 
# Show that there are no integers $x$ and $y$ such that $4x^3 - 7y^3 = 2003$.
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

  ✅ Proof attempt saved to: /root/assip2025/llama_proofs/LlamaProofs/mathd_algebra_107.lean

[86/244] Processing Problem: numbertheory_2pownm1prime_nprime
  Generating proof...
<|begin_of_text|>-- Formal statement 
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Show that if $n$ is a positive integer and $2^n - 1$ is prime, then $n$ is prime.-/
theorem numbertheory_2pownm1prime_nprime (n : ℕ) (h₀ : 0 < n) (h₁ : Nat.Prime (2 ^ n - 1)) : Nat.Prime n := by


-- Formal statement 
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Show that if $n$ is a positive integer and $2^n - 1$ is prime, then $n$ is prime.-/
theorem numbertheory_2pownm1prime_nprime (n : ℕ) (h₀ : 0 < n) (h₁ : Nat.Prime (2 ^ n - 1)) : Nat.Prime n := by
  intro n h₀ h₁
  have h₂ : 2 ^ n - 1 ≠ 0 := by
    intro h₃
    have h₄ : 2 ^ n = 1 := by
      have h₅ : 2 ^ n - 1 = 0 := by linarith [h₃]
      have h₆ : 2 ^ n = 1 -

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  ✅ Proof attempt saved to: /root/assip2025/llama_proofs/LlamaProofs/numbertheory_2pownm1prime_nprime.lean

[87/244] Processing Problem: mathd_algebra_412
  Generating proof...
<|begin_of_text|>-- Formal statement 
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The sum of two numbers is 25 and their difference is 11. What is the larger of the two numbers? Show that it is 18.-/
theorem mathd_algebra_412 (x y : ℝ) (h₀ : x + y = 25) (h₁ : x - y = 11) : x = 18 := by


-- Formal statement 
-- The sum of two numbers is 25 and their difference is 11. What is the larger of the two numbers? Show that it is 18.
-- 
-- # The sum of two numbers is 25 and their difference is 11. What is the larger of the two numbers? Show that it is 18.
-- # 
-- # ## Theorem
-- # ## 
-- # ## `mathd_algebra_412` 
-- # ## 
-- # ## Parameters
-- # ## 
-- # ## - `x` : ℝ
-- # ## - `y` : ℝ
-- # ## - `h₀` : `x + y = 25`
-- # ## - `h₁` : `x - y = 11`
-- # ## 
-- # ## R

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  ✅ Proof attempt saved to: /root/assip2025/llama_proofs/LlamaProofs/mathd_algebra_412.lean

[88/244] Processing Problem: amc12a_2013_p4
  Generating proof...
<|begin_of_text|>-- Formal statement 
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the value of $\frac{2^{2014}+2^{2012}}{2^{2014}-2^{2012}}?$

$ \textbf{(A)}\ -1\qquad\textbf{(B)}\ 1\qquad\textbf{(C)}\ \frac{5}{3}\qquad\textbf{(D)}\ 2013\qquad\textbf{(E)}\ 2^{4024} $ Show that it is \textbf{(C)} \frac{5}{3}.-/
theorem amc12a_2013_p4 : (2 ^ 2014 + 2 ^ 2012) / (2 ^ 2014 - 2 ^ 2012) = (5 : ℝ) / 3 := by


# Formal statement 
# 
# What is the value of $\frac{2^{2014}+2^{2012}}{2^{2014}-2^{2012}}?$

# $ \textbf{(A)}\ -1\qquad\textbf{(B)}\ 1\qquad\textbf{(C)}\ \frac{5}{3}\qquad\textbf{(D)}\ 2013\qquad\textbf{(E)}\ 2^{4024} $ 

# Show that it is \textbf{(C)} \frac{5}{3}.-/
# theorem amc12a_2013_p4 : (2 ^ 2014 + 2 ^ 2012) / (2 ^ 2014 - 2 ^ 2012) = (5 : ℝ) / 3 := by

# Forma